In [2]:
import serial
import time
import numpy as np

#pyqtgraph -> fast plotting
import pyqtgraph as pg
from pyqtgraph.Qt import QtGui
%gui qt5

import copy
from scipy.io import loadmat
import sys
np.set_printoptions(threshold=sys.maxsize)
from sklearn.cluster import DBSCAN
from sklearn.cluster import KMeans
import time
from pykalman import KalmanFilter
import pandas as pd

import hdbscan

C:\Users\hasna\Anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)
C:\Users\hasna\Anaconda3\lib\site-packages\sklearn\externals\six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


## Header

In [2]:
#load relevant header data
rawHeaderData = (loadmat('C:\\Users\\hasna\\Documents\\GitHub\\OccupancyDetection\\Data\\rawframeHeader.mat'))['rawframeHeader'][0]
#preprocessing of header
Documents/GitHub/OccupancyDetection/Data/Matlab Data
headerStream = np.array([])
for number in rawHeaderData[0]:
    headerStream = np.uint8(np.append(headerStream,number[0]))

In [ ]:
def readHeader(recieveHeader):
    headerContent = dict()
    index = 0
    
    headerContent['magicBytes'] = recieveHeader[index:index+8]
    index += 20
    
    headerContent['packetLength'] = recieveHeader[index:index+4].view(dtype=np.uint32)
    index += 4
        
    headerContent['frameNumber'] = recieveHeader[index:index+4].view(dtype=np.uint32)
    index += 24
    
    headerContent['numTLVs'] = recieveHeader[index:index+2].view(dtype=np.uint16)
    
    return headerContent

In [ ]:
def validateChecksum(recieveHeader):
    h = recieveHeader.view(dtype=np.uint16)
    a = np.array([sum(h)], dtype=np.uint32)
    b = np.array([sum(a.view(dtype=np.uint16))], dtype=np.uint16)
    CS = np.uint16(~(b))
    return CS

In [ ]:
#initialise variables
lostSync = False

#valid header variables and constant
magicBytes = np.array([2,1,4,3,6,5,8,7], dtype= 'uint8')

isMagicOk = False
isDataOk = False
gotHeader = False

frameHeaderLength = 52 #52 bytes long
tlvHeaderLengthInBytes = 8
pointLengthInBytes = 16
frameNumber = 1
targetFrameNumber = 0
targetLengthInBytes = 68

In [ ]:
header = readHeader(headerStream)
dataLength = int(header['packetLength'] - frameHeaderLength)

## TLV

In [10]:
tlvData = (loadmat('C:\\Users\\hasna\\Documents\\GitHub\\OccupancyDetection\\Data\\tlvData.mat'))['tlvData'][0][0]
tlvStream = np.frombuffer(tlvData, dtype = 'uint8')

### Point Cloud Header

In [ ]:
tlvStream = np.frombuffer(tlvData, dtype = 'uint8')

In [ ]:
#tlv header
index = 0
#tlv header parsing
tlvType = tlvStream[index:index+4].view(dtype=np.uint32)
tlvLength = tlvStream[index+4:index+8].view(dtype=np.uint32)

### Point Cloud

In [120]:
index += tlvHeaderLengthInBytes
tlvDataLength = tlvLength - tlvHeaderLengthInBytes

if tlvType == 6: 
    numberOfPoints = tlvDataLength/pointLengthInBytes
    p = tlvStream[index:index+tlvDataLength[0]].view(np.single)
    pointCloud = np.reshape(p,(4, int(numberOfPoints)),order="F")

    pointCloudConstrained = np.array([])
    if not(pointCloud is None):
        #constrain point cloud to within the effective sensor range
        #range 1 < x < 6
        #azimuth -50 deg to 50 deg
        #check whether corresponding range and azimuth data are within the constraints

        effectivePointCloud = np.array([])
        for index in range(0, len(pointCloud[0,:])):
            if (pointCloud[0,index] > 1 and pointCloud[0,index] < 6) and (pointCloud[1, index] > -50*np.pi/180 and pointCloud[1, index] < 50*np.pi/180):
                #concatenate columns to the new point cloud
                if len(effectivePointCloud) == 0:
                    effectivePointCloud = np.reshape(pointCloud[:, index], (4,1), order="F")
                else:
                    point = np.reshape(pointCloud[:, index], (4,1),order="F")
                    effectivePointCloud = np.hstack((effectivePointCloud, point))

        if len(effectivePointCloud) != 0:
            posX = np.multiply(effectivePointCloud[0,:], np.sin(effectivePointCloud[1,:]))
            posY = np.multiply(effectivePointCloud[0,:], np.cos(effectivePointCloud[1,:]))
            
            s.setData(posX,posY)
            QtGui.QApplication.processEvents() 


### Target List TLV Header

In [33]:
index += tlvDataLength
tlvType = tlvStream[index[0]:index[0]+4].view(dtype=np.uint32)
tlvLength = tlvStream[index[0]+4:index[0]+8].view(dtype=np.uint32)

In [37]:
print(tlvType)
print(tlvLength)
print(index)

[7]
[144]
[888]


### Target List

In [39]:
index += tlvHeaderLengthInBytes
targetListDataLength = tlvLength - tlvHeaderLengthInBytes

#setup target information objects
numberOfTargets = targetListDataLength/targetLengthInBytes
TID = np.zeros((1, int(numberOfTargets[0])), dtype = np.uint32) #tracking IDs
kinematicData = np.zeros((6, int(numberOfTargets[0])), dtype = np.single)
errorCovariance = np.zeros((9, int(numberOfTargets[0])), dtype = np.single)
gatingGain = np.zeros((1, int(numberOfTargets[0])), dtype = np.single)

In [40]:
targetIndex = 0
while targetIndex != int(numberOfTargets):
    TID[0][targetIndex] = tlvStream[index[0]:index[0]+4].view(dtype=np.uint32)
    kinematicData[:,targetIndex] = tlvStream[index[0]+4:index[0]+28].view(dtype=np.single)
    errorCovariance[:,targetIndex] = tlvStream[index[0]+28:index[0]+64].view(dtype=np.single)
    gatingGain[:,targetIndex] = tlvStream[index[0]+64:index[0]+68].view(dtype=np.single)
    index += targetLengthInBytes
    targetIndex += 1

## DBSCAN Implementation

In [2]:
#one person slow walk
#TLV Data Load
tlvData = (loadmat('C:\\Users\\hasna\\Documents\\GitHub\\OccupancyDetection\\Data\\Matlab Data\\slowWalk.mat'))['tlvStream'][0]

In [23]:
#extract noise
pureNoise = tlvData[0:100]
tlvData = pureNoise

In [370]:
#extract people walking 
walking = tlvData[200:500]
tlvData = walking

In [377]:
#two people slow walk
#TLV Data Load
tlvData = (loadmat('C:\\Users\\hasna\\Documents\\GitHub\\OccupancyDetection\\Data\\Matlab Data\\2PeopleMoving.mat'))['tlvStream'][0]

In [378]:
#three people walk
#TLV Data Load
tlvData = (loadmat('C:\\Users\\hasna\\Documents\\GitHub\\OccupancyDetection\\Data\\Matlab Data\\3PeopleWalking.mat'))['tlvStream'][0]

In [4]:
#initialise variables
lostSync = False

#valid header variables and constant
magicBytes = np.array([2,1,4,3,6,5,8,7], dtype= 'uint8')

isMagicOk = False
isDataOk = False
gotHeader = False

frameHeaderLength = 52 #52 bytes long
tlvHeaderLengthInBytes = 8
pointLengthInBytes = 16
frameNumber = 1
targetFrameNumber = 0
targetLengthInBytes = 68

#### Plotting Setup

In [4]:
app = QtGui.QApplication([])
pg.setConfigOption('background','w')

In [6]:
win = pg.GraphicsWindow(title="")
plot1 = win.addPlot()
plot1.setXRange(-6,6)
plot1.setYRange(0,6)
plot1.setLabel('left',text = 'Y position (m)')
plot1.setLabel('bottom', text= 'X position (m)')
s1 = plot1.plot([],[],pen=None,symbol='o')

plot2 = win1.addPlot()
plot2.setXRange(-6,6)
plot2.setYRange(0,6)
plot2.setLabel('left',text = 'Y position (m)')
plot2.setLabel('bottom', text= 'X position (m)')
s2 = plot2.plot([],[],pen=None,symbol='o')

NameError: name 'win1' is not defined

In [6]:
def createDBSCANDataset(posX, posY):
    #create DBSCAN dataset - find a more efficient way to do this
    dbscanDataSet = np.array([])
    for pointIndex in range(0, len(posX)):
        point = np.array([posX[pointIndex], posY[pointIndex]])
        if pointIndex == 0:
            dbscanDataSet = [point]
        else:
            dbscanDataSet = np.append(dbscanDataSet, [point], axis=0)
    return dbscanDataSet

In [7]:
#db is the output of the dbscan algorithm
def findClusters(db, dbscanDataSet):
    core_samples_mask = np.zeros_like(db.labels_, dtype=bool) #return an array of zeros with the same shape as labels
    core_samples_mask[db.core_sample_indices_] = True #place true where the index leads to a point which is in a cluster
    labels = db.labels_
    unique_labels = set(labels)
    xy = np.array([])
    clusterList = list()
    for label in unique_labels:
        if label == -1:
            continue
        class_member_mask = (labels == label) #mask all cluster members
        if len(xy) == 0:
            xy = dbscanDataSet[class_member_mask & core_samples_mask]
        else:    
            xy = np.concatenate((xy, dbscanDataSet[class_member_mask & core_samples_mask]),axis=0)

        clusterList.append(dbscanDataSet[class_member_mask & core_samples_mask])
    return clusterList, xy

In [8]:
def centroid(clusterList):
    centroids = np.array([])
    for cluster in clusterList:
        xMean = np.mean(cluster[:,0])
        yMean = np.mean(cluster[:,1])
        if len(centroids) == 0:
            centroids = np.array([xMean,yMean])
        else:
            centroids = np.concatenate((centroids, np.array([xMean,yMean])), axis=0)
    return centroids

In [226]:
centroidData = list();
for tlvStream in tlvData:
    tlvStream = np.frombuffer(tlvStream, dtype = 'uint8')
    
    #tlv header
    index = 0
    #tlv header parsing
    tlvType = tlvStream[index:index+4].view(dtype=np.uint32)
    tlvLength = tlvStream[index+4:index+8].view(dtype=np.uint32)
    
    index += tlvHeaderLengthInBytes
    tlvDataLength = tlvLength - tlvHeaderLengthInBytes

    if tlvType == 6: 
        numberOfPoints = tlvDataLength/pointLengthInBytes
        p = tlvStream[index:index+tlvDataLength[0]].view(np.single)
        pointCloud = np.reshape(p,(4, int(numberOfPoints)),order="F")

        if not(pointCloud is None):
            #constrain point cloud to within the effective sensor range
            #range 1 < x < 6
            #azimuth -50 deg to 50 deg
            #check whether corresponding range and azimuth data are within the constraints

            effectivePointCloud = np.array([])
            for index in range(0, len(pointCloud[0,:])):
                if (pointCloud[0,index] > 1 and pointCloud[0,index] < 6) \
                and (pointCloud[1, index] > -50*np.pi/180 \
                     and pointCloud[1, index] < 50*np.pi/180):
        
                    #concatenate columns to the new point cloud
                    if len(effectivePointCloud) == 0:
                        effectivePointCloud = np.reshape(pointCloud[:, index], (4,1), order="F")
                    else:
                        point = np.reshape(pointCloud[:, index], (4,1),order="F")
                        effectivePointCloud = np.hstack((effectivePointCloud, point))
                        

            if len(effectivePointCloud) != 0:
                posX = np.multiply(effectivePointCloud[0,:], np.sin(effectivePointCloud[1,:]))
                posY = np.multiply(effectivePointCloud[0,:], np.cos(effectivePointCloud[1,:]))
                
                #create DBSCAN dataset - find a more efficient way to do this
                dbscanDataSet = createDBSCANDataset(posX, posY)
                        
                #run DBSCAN
                db = DBSCAN(eps=0.15,metric='euclidean',min_samples=16).fit(dbscanDataSet) #loose constraints
                clusterList,xy1 = findClusters(db, dbscanDataSet)
                
                #eps - distance between clusters
                #min_samples - number of 
                
#                 centroids = centroid(clusterList)
#                 centroids = centroids.reshape(len(clusterList),2)
#                 centroidData.append(centroids)
             
                #plotting 
                if len(xy1) == 0:
                    s1.setData([],[])
                    s.setData(posX,posY)
                else:
                    for cluster in clusterList:
                        noClusters = len(clusterList)
                        if noClusters > 1:
                            print('WRONG')
                    s1.setData(posX,posY)
                    s2.setData(xy1[:, 0],xy1[:, 1])
                        
                QtGui.QApplication.processEvents() 
                time.sleep(0.04)
                


In [3]:
tlvData = loadmat('C:\\Users\\hasna\\Documents\\GitHub\\OccupancyDetection\\Data\\centroidData.mat')

In [5]:
centroid2 = loadmat('C:\\Users\\hasna\\Documents\\GitHub\\OccupancyDetection\\Data\\2peoplecentroidData.mat')

In [6]:
centroid2

{'__header__': b'MATLAB 5.0 MAT-file Platform: nt, Created on: Fri Jul 12 11:34:00 2019',
 '__version__': '1.0',
 '__globals__': [],
 'data': array([[array([], shape=(0, 0), dtype=float64),
         array([], shape=(0, 0), dtype=float64),
         array([], shape=(0, 0), dtype=float64),
         array([], shape=(0, 0), dtype=float64),
         array([], shape=(0, 0), dtype=float64),
         array([], shape=(0, 0), dtype=float64),
         array([], shape=(0, 0), dtype=float64),
         array([], shape=(0, 0), dtype=float64),
         array([], shape=(0, 0), dtype=float64),
         array([], shape=(0, 0), dtype=float64),
         array([], shape=(0, 0), dtype=float64),
         array([], shape=(0, 0), dtype=float64),
         array([], shape=(0, 0), dtype=float64),
         array([], shape=(0, 0), dtype=float64),
         array([], shape=(0, 0), dtype=float64),
         array([], shape=(0, 0), dtype=float64),
         array([], shape=(0, 0), dtype=float64),
         array([], shape=(

In [ ]:
savemat('centroidData.mat', {'data':centroidData})

In [4]:
tlvData

{'__header__': b'MATLAB 5.0 MAT-file Platform: nt, Created on: Thu Jul  4 16:00:45 2019',
 '__version__': '1.0',
 '__globals__': [],
 'data': array([[array([], shape=(0, 0), dtype=float64),
         array([], shape=(0, 0), dtype=float64),
         array([], shape=(0, 0), dtype=float64),
         array([], shape=(0, 0), dtype=float64),
         array([], shape=(0, 0), dtype=float64),
         array([], shape=(0, 0), dtype=float64),
         array([], shape=(0, 0), dtype=float64),
         array([], shape=(0, 0), dtype=float64),
         array([], shape=(0, 0), dtype=float64),
         array([], shape=(0, 0), dtype=float64),
         array([], shape=(0, 0), dtype=float64),
         array([], shape=(0, 0), dtype=float64),
         array([[2.0187733, 2.0371823]], dtype=float32),
         array([], shape=(0, 0), dtype=float64),
         array([], shape=(0, 0), dtype=float64),
         array([], shape=(0, 0), dtype=float64),
         array([], shape=(0, 0), dtype=float64),
         array([],

### Variable DBSCAN

In [23]:
win = pg.GraphicsWindow(title="Occupancy Detection GUI")
plot1 = win.addPlot()
plot1.setXRange(-6,6)
plot1.setYRange(0,6)
plot1.setLabel('left',text = 'Y position (m)')
plot1.setLabel('bottom', text= 'X position (m)')
s1 = plot1.plot([],[],pen=None,symbol='o')

plot2 = win.addPlot()
plot2.setXRange(-6,6)
plot2.setYRange(0,6)
plot2.setLabel('left',text = 'Y position (m)')
plot2.setLabel('bottom', text= 'X position (m)')
s2 = plot2.plot([],[],pen=None,symbol='o')

text = win.addLabel(text='Occpuancy Estimate: 0')

In [349]:
#change occupancy estimate information
win.removeItem(text)
text = win.addLabel(text='Occpuancy Estimate: 1')

In [9]:
#db is the output of the dbscan algorithm
def modifiedFindClusters(db, dbscanDataSet):
    clusterList = list()
    numClusters = max(db.labels_)+1
    for i in range(0, numClusters):
        clusterMask = db.labels_ == i
        cluster = dbscanDataSet[clusterMask]
        if len(cluster) > 0:
            clusterList.append(cluster)
    
    return clusterList

In [10]:
def centroid(clusterList):
    centroids = np.array([])
    for cluster in clusterList:
        xMean = np.mean(cluster['X'])
        yMean = np.mean(cluster['Y'])
        if len(centroids) == 0:
            centroids = np.array([xMean,yMean])
        else:
            centroids = np.concatenate((centroids, np.array([xMean,yMean])), axis=0)
    return centroids

In [11]:
def assembleFilterPointArray(clusterList):
    pointArray = np.array([])
    for cluster in clusterList:
        if len(pointArray) == 0:
            pointArray = cluster.values
        else:
            pointArray = np.concatenate([pointArray, cluster.values])
    return pointArray

In [25]:
centroidData = list()
entireClusterChain = list()

for tlvStream in tlvData:
    tlvStream = np.frombuffer(tlvStream, dtype = 'uint8')
    
    #tlv header
    index = 0
    #tlv header parsing
    tlvType = tlvStream[index:index+4].view(dtype=np.uint32)
    tlvLength = tlvStream[index+4:index+8].view(dtype=np.uint32)
    
    index += tlvHeaderLengthInBytes
    tlvDataLength = tlvLength - tlvHeaderLengthInBytes

    if tlvType == 6: 
        numberOfPoints = tlvDataLength/pointLengthInBytes
        p = tlvStream[index:index+tlvDataLength[0]].view(np.single)
        pointCloud = np.reshape(p,(4, int(numberOfPoints)),order="F")

        if not(pointCloud is None):
            #constrain point cloud to within the effective sensor range
            #range 1 < x < 6
            #azimuth -50 deg to 50 deg
            #check whether corresponding range and azimuth data are within the constraints

            effectivePointCloud = np.array([])
            for index in range(0, len(pointCloud[0,:])):
                if (pointCloud[0,index] > 1 and pointCloud[0,index] < 6) \
                and (pointCloud[1, index] > -50*np.pi/180 \
                     and pointCloud[1, index] < 50*np.pi/180):
        
                    #concatenate columns to the new point cloud
                    if len(effectivePointCloud) == 0:
                        effectivePointCloud = np.reshape(pointCloud[:, index], (4,1), order="F")
                    else:
                        point = np.reshape(pointCloud[:, index], (4,1),order="F")
                        effectivePointCloud = np.hstack((effectivePointCloud, point))
                        

            if len(effectivePointCloud) != 0:
                #effective point cloud
                posX = np.multiply(effectivePointCloud[0,:], np.sin(effectivePointCloud[1,:]))
                posY = np.multiply(effectivePointCloud[0,:], np.cos(effectivePointCloud[1,:]))
                
                #create DBSCAN dataset - find a more efficient way to do this
                dbscanDataSet = pd.DataFrame({'X':posX,'Y':posY})
                s1.setData(dbscanDataSet.values)
                
                #run DBSCAN
                db13 = DBSCAN(eps=0.15,metric='euclidean',min_samples=10).fit(dbscanDataSet.values)
                db36= DBSCAN(eps=0.25,metric='euclidean',min_samples=10).fit(dbscanDataSet.values)
                
                clusterList13 = modifiedFindClusters(db13, dbscanDataSet)
                clusterList36 = modifiedFindClusters(db36, dbscanDataSet)
            
                dbPoints13 = assembleFilterPointArray(clusterList13)
                dbPoints36 = assembleFilterPointArray(clusterList36)
                
                df13 = pd.DataFrame([])
                df36 = pd.DataFrame([])
                if len(dbPoints36) > 0:
                    df36 = pd.DataFrame(dbPoints36)
                    df36Mask = df36[1] >=3 #select points where y > 3
                    df36 = df36[df36Mask]
                if len(dbPoints13) > 0:
                    df13 = pd.DataFrame(dbPoints13)
                    df13Mask = df13[1] < 3 #select points where y < 3
                    df13 = df13[df13Mask]
                
                #join the two areas and then perform AgglomerativeClustering
                fullClusterSet = pd.concat([df13, df36])
                s2.setData(fullClusterSet.values)
                if len(fullClusterSet.values) > 0:
                    hdbClusterer = hdbscan.HDBSCAN()
                    hdb = hdbClusterer.fit(fullClusterSet.values)
                
                time.sleep(0.1)

KeyboardInterrupt: 

In [15]:
len(fullClusterSet)

0

In [328]:
df36 = pd.DataFrame(dbPoints36)
df36Mask = df36[1] > 3 #select points where y > 3
df36 = df36[df36Mask]


           0         1
0   1.429141  4.210114
1   1.476256  4.348909
2   1.491961  4.395175
3   1.507666  4.441440
4   1.276442  4.207867
5   1.290625  4.254621
6   1.304808  4.301375
7   1.318990  4.348129
8   1.333173  4.394883
9   1.347356  4.441637
10  1.361539  4.488391
11  1.125436  4.200184
12  1.138081  4.247377
13  1.150727  4.294571
14  1.163372  4.341763
15  1.176017  4.388957
16  1.188663  4.436150
17  1.201308  4.483343
18  1.213953  4.530536
19  0.998502  4.282340
20  1.009596  4.329922
21  1.020691  4.377503
22  1.031785  4.425085
23  1.042880  4.472667
24  1.053974  4.520248
25  1.065069  4.567830
26  0.886448  4.456475
27  0.905511  4.552313
28  0.915043  4.600232
           0         1
0   1.429141  4.210114
1   1.476256  4.348909
2   1.491961  4.395175
3   1.507666  4.441440
4   1.276442  4.207867
5   1.290625  4.254621
6   1.304808  4.301375
7   1.318990  4.348129
8   1.333173  4.394883
9   1.347356  4.441637
10  1.361539  4.488391
11  1.125436  4.200184
12  1.13808

### Two Level DBSCAN Algorithm

In [ ]:
for tlvStream in tlvData:
    tlvStream = np.frombuffer(tlvStream, dtype = 'uint8')
    
    #tlv header
    index = 0
    #tlv header parsing
    tlvType = tlvStream[index:index+4].view(dtype=np.uint32)
    tlvLength = tlvStream[index+4:index+8].view(dtype=np.uint32)
    
    index += tlvHeaderLengthInBytes
    tlvDataLength = tlvLength - tlvHeaderLengthInBytes

    if tlvType == 6: 
        numberOfPoints = tlvDataLength/pointLengthInBytes
        p = tlvStream[index:index+tlvDataLength[0]].view(np.single)
        pointCloud = np.reshape(p,(4, int(numberOfPoints)),order="F")

        if not(pointCloud is None):
            #constrain point cloud to within the effective sensor range
            #range 1 < x < 6
            #azimuth -50 deg to 50 deg
            #check whether corresponding range and azimuth data are within the constraints

            effectivePointCloud = np.array([])
            for index in range(0, len(pointCloud[0,:])):
                if (pointCloud[0,index] > 1 and pointCloud[0,index] < 6) and (pointCloud[1, index] > -50*np.pi/180 and pointCloud[1, index] < 50*np.pi/180):
                    #concatenate columns to the new point cloud
                    if len(effectivePointCloud) == 0:
                        effectivePointCloud = np.reshape(pointCloud[:, index], (4,1), order="F")
                    else:
                        point = np.reshape(pointCloud[:, index], (4,1),order="F")
                        effectivePointCloud = np.hstack((effectivePointCloud, point))

            if len(effectivePointCloud) != 0:
                posX = np.multiply(effectivePointCloud[0,:], np.sin(effectivePointCloud[1,:]))
                posY = np.multiply(effectivePointCloud[0,:], np.cos(effectivePointCloud[1,:]))
                
                #create DBSCAN dataset - find a more efficient way to do this
                dbscanDataSet = createDBSCANDataset(posX, posY)
                        
                #run DBSCAN
                dbLoose = DBSCAN(eps=0.4,metric='euclidean',min_samples=10).fit(dbscanDataSet) #loose constraints
                clusterList,xy1 = findClusters(dbLoose, dbscanDataSet)
                if len(clusterList) != 0:
                    dbTight = DBSCAN(eps=0.2,metric='euclidean',min_samples=18).fit(xy1) #loose constraints
                    clusterList,xy2 = findClusters(dbTight, xy1)
                    if len(xy2) != 0:
                        s1.setData(xy2[:, 0],xy2[:, 1])
                        QtGui.QApplication.processEvents() 
                    s.setData(xy1[:, 0],xy1[:, 1])
                    QtGui.QApplication.processEvents() 
#                     time.sleep(0.1)
                    enter = input("enter: ")

## HBDSCAN

In [91]:
def hdbscanFindClusters(positions,clusterer):
    #each row in the positions dataframe is data sample
    #clusterer contains HDBSCAN object
    clusterList = list()
    numClusters = max(clusterer.labels_)+1
    print(numClusters)
    for i in range(0, numClusters):
        clusterMask = clusterer.labels_ == i
        cluster = positions.values[clusterMask]
        clusterList.append(cluster)
    return clusterList

In [12]:
clusterer = hdbscan.HDBSCAN()
centroidData = list();

for tlvStream in tlvData:
    tlvStream = np.frombuffer(tlvStream, dtype = 'uint8')
    
    #tlv header
    index = 0
    #tlv header parsing
    tlvType = tlvStream[index:index+4].view(dtype=np.uint32)
    tlvLength = tlvStream[index+4:index+8].view(dtype=np.uint32)
    
    index += tlvHeaderLengthInBytes
    tlvDataLength = tlvLength - tlvHeaderLengthInBytes

    if tlvType == 6: 
        numberOfPoints = tlvDataLength/pointLengthInBytes
        p = tlvStream[index:index+tlvDataLength[0]].view(np.single)
        pointCloud = np.reshape(p,(4, int(numberOfPoints)),order="F")

        if not(pointCloud is None):
            #constrain point cloud to within the effective sensor range
            #range 1 < x < 6
            #azimuth -50 deg to 50 deg
            #check whether corresponding range and azimuth data are within the constraints

            effectivePointCloud = np.array([])
            for index in range(0, len(pointCloud[0,:])):
                if (pointCloud[0,index] > 1 and pointCloud[0,index] < 6) \
                and (pointCloud[1, index] > -50*np.pi/180 \
                     and pointCloud[1, index] < 50*np.pi/180):
        
                    #concatenate columns to the new point cloud
                    if len(effectivePointCloud) == 0:
                        effectivePointCloud = np.reshape(pointCloud[:, index], (4,1), order="F")
                    else:
                        point = np.reshape(pointCloud[:, index], (4,1),order="F")
                        effectivePointCloud = np.hstack((effectivePointCloud, point))
                        

            if len(effectivePointCloud) != 0:
                posX = np.multiply(effectivePointCloud[0,:], np.sin(effectivePointCloud[1,:]))
                posY = np.multiply(effectivePointCloud[0,:], np.cos(effectivePointCloud[1,:]))
#                 s.setData(posX, posY)
                positions = pd.DataFrame({'X':posX,'Y':posY})
                #positions.values will then have the x,y positions array that we want
                if len(positions.values) >= 10:
                    clusterer = hdbscan.HDBSCAN(min_cluster_size=5)
                    clusterer.fit(positions.values)
                    print(max(clusterer.labels_)+1)



0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
0
0
2
0
0
0
0
0
0
3
2
0
2
2
2
2
2
2
0
2
2
2
2
2
2
3
2
2
2
2
2
2
2
2
2
2
0
0
0
2
0
2
2
2
2
2
2
3
2
2
2
2
2
2
2
2
2
2
0
2
0
0
0
0
0
0
2
0
2
2
2
2
2
2
0
2
0
0
2
0
0
0
0
0
0
0
2
0
0
2
0
0
0
0
2
2
2
2
2
2
2
2
2
2
2
2
3
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
0
2
2
0
2
2
0
2
0
0
2
0
0
0
0
2
2
0
0
0
0
0
2
0
0
0
0
0
2
2
0
0
0
2
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
2
0
2
2
2
2
2
0
0
0
0
0
0
0
0
0
2
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
2
2
2
2
0
2
2
0
2
0
2
2
2
0
2
2
2
2
2
2
2
2
2
2
2
2
0
2
0
0
0
2
2
2
2
2
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
0
2
2
2
0
0
2
0
0
0
0
0
2
0
0
0
0
2
0
0
0
0
0
0
0
0
2
0
2
0
0
2
2
0
2
2
2
2
0
2
2
2
2
2
2
2
2
2
2
2
2
0
0
2
2
2
0
0
0
2
0
2
2
2
2
2
0
0
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
2
2
2
2
2
2
2
2
2
2
2


In [7]:
clusterer.labels_

array([-1, -1], dtype=int64)

array([2.0383162, 2.103917 ], dtype=float32)

array([2.0383162, 1.970533 ], dtype=float32)